# Life Expentancy Determinator by Countries Worldwide

In [6]:
# Initial imports
import pandas as pd
!pip install hvplot
import hvplot.pandas
!pip install path
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
from sklearn.impute import SimpleImputer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


# Preprocessing the Data for PCA
* Here we will need load, clean up, and scale the dataset
* Current cleanup steps we can determine are:
    * Remove features that do not apply to all countries
    * Focus on a subset of more recent years (maybe from 2010) forward as data is more prevalent in those years
    * Remove features that do not have hardy data
    * Determine if we should rationalize certain features where data does not exist
    * Group data by years then take the mean so that data is one value per feature

In [13]:
from google.colab import files
uploaded = files.upload()

Saving worldbank_data.csv to worldbank_data.csv


In [14]:
file_path = "worldbank_data.csv"
worldbank_df = pd.read_csv(file_path)
worldbank_df

,Unnamed: 0,SP.RUR.TOTL.ZG,SP.DYN.LE00.MA.IN,SP.DYN.CDRT.IN,SE.SEC.AGES,TT.PRI.MRCH.XD.WD,TM.VAL.MRCH.HI.ZS,SP.URB.TOTL.IN.ZS,SP.POP.TOTL,NY.ADJ.AEDU.GN.ZS,...,SE.SEC.DURS,NY.ADJ.DCO2.CD,TX.VAL.MRCH.XD.WD,TM.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TM.UVI.MRCH.XD.WD,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.MKTP.CN
0,ABW,1.165446,72.533619,8.004524,12.0,107.455250,84.024361,44.087857,1.012436e+05,5.534013,...,5.000000,4.821349e+07,79.650577,4.783515,1.125688e+08,216.658854,NaN,NaN,NaN,NaN
1,ASM,0.219893,NaN,NaN,NaN,145.250894,71.802046,87.698762,5.704352e+04,11.760709,...,NaN,1.305928e+07,114.155453,0.002126,4.184260e+07,143.513039,NaN,NaN,NaN,NaN
2,BEL,-1.441178,77.380952,9.809524,12.0,97.331406,84.902486,97.634000,1.088938e+07,5.853018,...,6.000000,2.914562e+09,201.751129,0.126156,3.792406e+11,159.111986,4.421130e+11,40354.739298,3.329685e+04,3.646237e+11
3,BRB,0.498354,76.756810,8.353000,11.0,108.709459,82.995957,32.113905,2.808053e+05,5.273782,...,5.000000,3.839135e+07,151.921145,0.568932,3.391027e+08,156.621750,4.297310e+09,15267.202063,3.053440e+04,8.594619e+09
4,AFG,2.877665,59.468381,8.529857,13.0,144.139332,13.475310,23.813571,2.976380e+07,2.633482,...,6.000000,1.917496e+08,332.657639,17.256922,4.615880e+08,135.988659,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,UKR,-0.939745,64.564762,15.342857,10.0,97.418236,45.066431,68.436762,4.619452e+07,5.381339,...,7.000000,7.684421e+09,295.484613,0.344069,4.288368e+10,199.027240,1.165848e+11,2599.344442,3.400556e+04,1.480505e+12
209,URY,-2.673514,72.779762,9.386048,12.0,106.540275,29.258109,94.139095,3.373326e+06,3.663434,...,6.000000,1.894705e+08,250.734485,4.982259,5.763423e+09,132.264065,3.875424e+10,11415.997493,2.947178e+05,1.003606e+12
210,UZB,1.192000,66.939524,5.033333,11.0,156.182226,37.723131,49.581048,2.885206e+07,9.374764,...,7.772727,3.061872e+09,306.189384,0.006456,7.316529e+09,134.589546,4.345603e+10,1438.250604,4.762089e+06,1.520066e+14
211,ZMB,2.088479,52.597905,10.936762,14.0,159.612380,29.054215,39.449000,1.393321e+07,2.556854,...,5.000000,1.051217e+08,617.622311,0.695017,5.549497e+09,144.208141,1.674368e+10,1144.799404,7.938876e+03,1.236047e+11


In [15]:
worldbank_df.count()

Unnamed: 0           213
SP.RUR.TOTL.ZG       201
SP.DYN.LE00.MA.IN    196
SP.DYN.CDRT.IN       198
SE.SEC.AGES          201
TT.PRI.MRCH.XD.WD    197
TM.VAL.MRCH.HI.ZS    194
SP.URB.TOTL.IN.ZS    210
SP.POP.TOTL          212
NY.ADJ.AEDU.GN.ZS    196
TM.VAL.MRCH.WL.CD    194
SE.PRM.DURS          204
NY.ADJ.DNGY.CD       200
TX.VAL.MRCH.RS.ZS    194
TX.VAL.MRCH.HI.ZS    194
TM.QTY.MRCH.XD.WD    197
SP.RUR.TOTL.ZS       210
SP.POP.GROW          212
SP.DYN.TFRT.IN       196
PA.NUS.ATLS          196
SP.URB.GROW          210
PA.NUS.FCRF          197
AG.LND.TOTL.K2       211
TX.QTY.MRCH.XD.WD    198
SE.PRM.AGES          204
SP.URB.TOTL          210
NY.ADJ.DMIN.CD       213
EG.ELC.ACCS.UR.ZS    204
TX.UVI.MRCH.XD.WD    196
SP.DYN.CBRT.IN       198
SP.DYN.LE00.FE.IN    196
SH.TBS.INCD          199
EG.ELC.ACCS.ZS       207
TM.VAL.MRCH.XD.WD    198
SP.RUR.TOTL          210
SP.DYN.LE00.IN       196
AG.LND.FRST.K2       203
EN.POP.DNST          211
AG.LND.FRST.ZS       203
SE.SEC.DURS          201


In [16]:
worldbank_df.rename(columns = {'Unnamed: 0':'Country'}, inplace = True)
worldbank_df.head()

,Country,SP.RUR.TOTL.ZG,SP.DYN.LE00.MA.IN,SP.DYN.CDRT.IN,SE.SEC.AGES,TT.PRI.MRCH.XD.WD,TM.VAL.MRCH.HI.ZS,SP.URB.TOTL.IN.ZS,SP.POP.TOTL,NY.ADJ.AEDU.GN.ZS,...,SE.SEC.DURS,NY.ADJ.DCO2.CD,TX.VAL.MRCH.XD.WD,TM.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TM.UVI.MRCH.XD.WD,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.MKTP.CN
0,ABW,1.165446,72.533619,8.004524,12.0,107.455250,84.024361,44.087857,1.012436e+05,5.534013,...,5.0,4.821349e+07,79.650577,4.783515,1.125688e+08,216.658854,NaN,NaN,NaN,NaN
1,ASM,0.219893,NaN,NaN,NaN,145.250894,71.802046,87.698762,5.704352e+04,11.760709,...,NaN,1.305928e+07,114.155453,0.002126,4.184260e+07,143.513039,NaN,NaN,NaN,NaN
2,BEL,-1.441178,77.380952,9.809524,12.0,97.331406,84.902486,97.634000,1.088938e+07,5.853018,...,6.0,2.914562e+09,201.751129,0.126156,3.792406e+11,159.111986,4.421130e+11,40354.739298,33296.850099,3.646237e+11
3,BRB,0.498354,76.756810,8.353000,11.0,108.709459,82.995957,32.113905,2.808053e+05,5.273782,...,5.0,3.839135e+07,151.921145,0.568932,3.391027e+08,156.621750,4.297310e+09,15267.202063,30534.404126,8.594619e+09
4,AFG,2.877665,59.468381,8.529857,13.0,144.139332,13.475310,23.813571,2.976380e+07,2.633482,...,6.0,1.917496e+08,332.657639,17.256922,4.615880e+08,135.988659,NaN,NaN,NaN,NaN


In [17]:
# Create a new DataFrame that holds only the countries names.
country_names_df = worldbank_df.filter(['Country'], axis=1)
#country_names_df = country_names_df.set_index('Symbol')
country_names_df

,Country
0,ABW
1,ASM
2,BEL
3,BRB
4,AFG
...,...
208,UKR
209,URY
210,UZB
211,ZMB


In [18]:
# Drop the 'Country' column since it's not going to be used on the clustering algorithm.
worldbank_df.drop('Country', axis=1, inplace=True)
#worldbank_df = worldbank_df.set_index('Symbol')
worldbank_df

,SP.RUR.TOTL.ZG,SP.DYN.LE00.MA.IN,SP.DYN.CDRT.IN,SE.SEC.AGES,TT.PRI.MRCH.XD.WD,TM.VAL.MRCH.HI.ZS,SP.URB.TOTL.IN.ZS,SP.POP.TOTL,NY.ADJ.AEDU.GN.ZS,TM.VAL.MRCH.WL.CD,...,SE.SEC.DURS,NY.ADJ.DCO2.CD,TX.VAL.MRCH.XD.WD,TM.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TM.UVI.MRCH.XD.WD,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.MKTP.CN
0,1.165446,72.533619,8.004524,12.0,107.455250,84.024361,44.087857,1.012436e+05,5.534013,1.087467e+09,...,5.000000,4.821349e+07,79.650577,4.783515,1.125688e+08,216.658854,NaN,NaN,NaN,NaN
1,0.219893,NaN,NaN,NaN,145.250894,71.802046,87.698762,5.704352e+04,11.760709,1.792657e+08,...,NaN,1.305928e+07,114.155453,0.002126,4.184260e+07,143.513039,NaN,NaN,NaN,NaN
2,-1.441178,77.380952,9.809524,12.0,97.331406,84.902486,97.634000,1.088938e+07,5.853018,3.630866e+11,...,6.000000,2.914562e+09,201.751129,0.126156,3.792406e+11,159.111986,4.421130e+11,40354.739298,3.329685e+04,3.646237e+11
3,0.498354,76.756810,8.353000,11.0,108.709459,82.995957,32.113905,2.808053e+05,5.273782,1.516150e+09,...,5.000000,3.839135e+07,151.921145,0.568932,3.391027e+08,156.621750,4.297310e+09,15267.202063,3.053440e+04,8.594619e+09
4,2.877665,59.468381,8.529857,13.0,144.139332,13.475310,23.813571,2.976380e+07,2.633482,5.220148e+09,...,6.000000,1.917496e+08,332.657639,17.256922,4.615880e+08,135.988659,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,-0.939745,64.564762,15.342857,10.0,97.418236,45.066431,68.436762,4.619452e+07,5.381339,4.894700e+10,...,7.000000,7.684421e+09,295.484613,0.344069,4.288368e+10,199.027240,1.165848e+11,2599.344442,3.400556e+04,1.480505e+12
209,-2.673514,72.779762,9.386048,12.0,106.540275,29.258109,94.139095,3.373326e+06,3.663434,7.075817e+09,...,6.000000,1.894705e+08,250.734485,4.982259,5.763423e+09,132.264065,3.875424e+10,11415.997493,2.947178e+05,1.003606e+12
210,1.192000,66.939524,5.033333,11.0,156.182226,37.723131,49.581048,2.885206e+07,9.374764,1.363325e+10,...,7.772727,3.061872e+09,306.189384,0.006456,7.316529e+09,134.589546,4.345603e+10,1438.250604,4.762089e+06,1.520066e+14
211,2.088479,52.597905,10.936762,14.0,159.612380,29.054215,39.449000,1.393321e+07,2.556854,5.404199e+09,...,5.000000,1.051217e+08,617.622311,0.695017,5.549497e+09,144.208141,1.674368e+10,1144.799404,7.938876e+03,1.236047e+11


In [19]:
# Impute NaN values using SKlearn Imputer
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(worldbank_df[['NY.GDP.MKTP.CN','NY.GDP.PCAP.CN','NY.GDP.PCAP.CD','NY.GDP.MKTP.CD','TM.UVI.MRCH.XD.WD','TX.VAL.MRCH.WL.CD','TM.VAL.MRCH.RS.ZS','TX.VAL.MRCH.XD.WD','NY.ADJ.DCO2.CD','SE.SEC.DURS','AG.LND.FRST.ZS','EN.POP.DNST','AG.LND.FRST.K2','SP.DYN.LE00.IN','SP.RUR.TOTL','TM.VAL.MRCH.XD.WD','EG.ELC.ACCS.ZS','SH.TBS.INCD','SP.DYN.LE00.FE.IN','SP.DYN.CBRT.IN','TX.UVI.MRCH.XD.WD','EG.ELC.ACCS.UR.ZS','NY.ADJ.DMIN.CD','SP.URB.TOTL','SE.PRM.AGES','TX.QTY.MRCH.XD.WD','AG.LND.TOTL.K2','PA.NUS.FCRF','SP.URB.GROW','PA.NUS.ATLS','SP.DYN.TFRT.IN','SP.POP.GROW','SP.RUR.TOTL.ZS','TM.QTY.MRCH.XD.WD','TX.VAL.MRCH.HI.ZS','TX.VAL.MRCH.RS.ZS','NY.ADJ.DNGY.CD','SE.PRM.DURS','TM.VAL.MRCH.WL.CD','NY.ADJ.AEDU.GN.ZS','SP.POP.TOTL','SP.URB.TOTL.IN.ZS','TM.VAL.MRCH.HI.ZS','TT.PRI.MRCH.XD.WD','SE.SEC.AGES','SP.DYN.CDRT.IN','SP.DYN.LE00.MA.IN','SP.RUR.TOTL.ZG'
]])
worldbank_df[['NY.GDP.MKTP.CN','NY.GDP.PCAP.CN','NY.GDP.PCAP.CD','NY.GDP.MKTP.CD','TM.UVI.MRCH.XD.WD','TX.VAL.MRCH.WL.CD','TM.VAL.MRCH.RS.ZS','TX.VAL.MRCH.XD.WD','NY.ADJ.DCO2.CD','SE.SEC.DURS','AG.LND.FRST.ZS','EN.POP.DNST','AG.LND.FRST.K2','SP.DYN.LE00.IN','SP.RUR.TOTL','TM.VAL.MRCH.XD.WD','EG.ELC.ACCS.ZS','SH.TBS.INCD','SP.DYN.LE00.FE.IN','SP.DYN.CBRT.IN','TX.UVI.MRCH.XD.WD','EG.ELC.ACCS.UR.ZS','NY.ADJ.DMIN.CD','SP.URB.TOTL','SE.PRM.AGES','TX.QTY.MRCH.XD.WD','AG.LND.TOTL.K2','PA.NUS.FCRF','SP.URB.GROW','PA.NUS.ATLS','SP.DYN.TFRT.IN','SP.POP.GROW','SP.RUR.TOTL.ZS','TM.QTY.MRCH.XD.WD','TX.VAL.MRCH.HI.ZS','TX.VAL.MRCH.RS.ZS','NY.ADJ.DNGY.CD','SE.PRM.DURS','TM.VAL.MRCH.WL.CD','NY.ADJ.AEDU.GN.ZS','SP.POP.TOTL','SP.URB.TOTL.IN.ZS','TM.VAL.MRCH.HI.ZS','TT.PRI.MRCH.XD.WD','SE.SEC.AGES','SP.DYN.CDRT.IN','SP.DYN.LE00.MA.IN','SP.RUR.TOTL.ZG'
]] = imputer.transform(worldbank_df[['NY.GDP.MKTP.CN','NY.GDP.PCAP.CN','NY.GDP.PCAP.CD','NY.GDP.MKTP.CD','TM.UVI.MRCH.XD.WD','TX.VAL.MRCH.WL.CD','TM.VAL.MRCH.RS.ZS','TX.VAL.MRCH.XD.WD','NY.ADJ.DCO2.CD','SE.SEC.DURS','AG.LND.FRST.ZS','EN.POP.DNST','AG.LND.FRST.K2','SP.DYN.LE00.IN','SP.RUR.TOTL','TM.VAL.MRCH.XD.WD','EG.ELC.ACCS.ZS','SH.TBS.INCD','SP.DYN.LE00.FE.IN','SP.DYN.CBRT.IN','TX.UVI.MRCH.XD.WD','EG.ELC.ACCS.UR.ZS','NY.ADJ.DMIN.CD','SP.URB.TOTL','SE.PRM.AGES','TX.QTY.MRCH.XD.WD','AG.LND.TOTL.K2','PA.NUS.FCRF','SP.URB.GROW','PA.NUS.ATLS','SP.DYN.TFRT.IN','SP.POP.GROW','SP.RUR.TOTL.ZS','TM.QTY.MRCH.XD.WD','TX.VAL.MRCH.HI.ZS','TX.VAL.MRCH.RS.ZS','NY.ADJ.DNGY.CD','SE.PRM.DURS','TM.VAL.MRCH.WL.CD','NY.ADJ.AEDU.GN.ZS','SP.POP.TOTL','SP.URB.TOTL.IN.ZS','TM.VAL.MRCH.HI.ZS','TT.PRI.MRCH.XD.WD','SE.SEC.AGES','SP.DYN.CDRT.IN','SP.DYN.LE00.MA.IN','SP.RUR.TOTL.ZG'
]])
worldbank_df.head()

,SP.RUR.TOTL.ZG,SP.DYN.LE00.MA.IN,SP.DYN.CDRT.IN,SE.SEC.AGES,TT.PRI.MRCH.XD.WD,TM.VAL.MRCH.HI.ZS,SP.URB.TOTL.IN.ZS,SP.POP.TOTL,NY.ADJ.AEDU.GN.ZS,TM.VAL.MRCH.WL.CD,...,SE.SEC.DURS,NY.ADJ.DCO2.CD,TX.VAL.MRCH.XD.WD,TM.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TM.UVI.MRCH.XD.WD,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.MKTP.CN
0,1.165446,72.533619,8.004524,12.000000,107.455250,84.024361,44.087857,1.012436e+05,5.534013,1.087467e+09,...,5.000000,4.821349e+07,79.650577,4.783515,1.125688e+08,216.658854,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14
1,0.219893,67.868398,8.191021,11.829812,145.250894,71.802046,87.698762,5.704352e+04,11.760709,1.792657e+08,...,6.335649,1.305928e+07,114.155453,0.002126,4.184260e+07,143.513039,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14
2,-1.441178,77.380952,9.809524,12.000000,97.331406,84.902486,97.634000,1.088938e+07,5.853018,3.630866e+11,...,6.000000,2.914562e+09,201.751129,0.126156,3.792406e+11,159.111986,4.421130e+11,40354.739298,3.329685e+04,3.646237e+11
3,0.498354,76.756810,8.353000,11.000000,108.709459,82.995957,32.113905,2.808053e+05,5.273782,1.516150e+09,...,5.000000,3.839135e+07,151.921145,0.568932,3.391027e+08,156.621750,4.297310e+09,15267.202063,3.053440e+04,8.594619e+09
4,2.877665,59.468381,8.529857,13.000000,144.139332,13.475310,23.813571,2.976380e+07,2.633482,5.220148e+09,...,6.000000,1.917496e+08,332.657639,17.256922,4.615880e+08,135.988659,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14


In [20]:
x=worldbank_df

In [21]:
# Standardize the data with StandardScaler().
scaler = StandardScaler().fit(x)
x_scaled = scaler.transform(x)
x_scaled

array([[ 5.45049185e-01,  5.73317031e-01, -6.40961979e-02, ...,
         0.00000000e+00,  5.61426036e-17,  1.95526625e-17],
       [-1.20700362e-01, -1.74639634e-15,  0.00000000e+00, ...,
         0.00000000e+00,  5.61426036e-17,  1.95526625e-17],
       [-1.29023630e+00,  1.16901415e+00,  5.56254328e-01, ...,
         1.18293400e+00, -2.06079703e-01, -1.47345113e-01],
       ...,
       [ 5.63745418e-01, -1.14150939e-01, -1.08524832e+00, ...,
        -6.30468225e-01,  3.64048599e-01,  4.24152154e-02],
       [ 1.19494306e+00, -1.87661713e+00,  9.43668592e-01, ...,
        -6.44142251e-01, -2.09136995e-01, -1.47646717e-01],
       [ 5.46878356e-01, -2.21029155e+00,  1.68966035e+00, ...,
        -6.54772485e-01, -2.09468459e-01, -1.47706779e-01]])

# Reducing Data Dimensions Using PCA

In [22]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca = pca.fit(x_scaled)
X_pca = pca.transform(x_scaled)
print(pca.explained_variance_ratio_)

[0.22419894 0.14761325 0.07895621]


In [23]:
# Create a DataFrame with the three principal components.  Will need to test how many PC's is best fit, could be more than 3.
pcs_df = pd.DataFrame(data=X_pca, columns=["PC1", "PC2", "PC3"])
pcs_df.head(10)

,PC1,PC2,PC3
0,-1.458722,-1.883285,-0.504422
1,-1.415482,-1.809381,-0.423181
2,-4.289612,-0.851653,-0.802823
3,-2.219694,-1.981160,-0.642262
4,5.584652,1.086053,-0.171742
5,-1.414161,-1.307499,0.285723
6,-2.182922,-1.058004,0.939800
7,-1.026563,-1.088662,-0.642386
8,5.604188,1.891779,-0.281637
9,-0.932873,-0.829744,-0.138219


### Clustering Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [24]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1,11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=?`

In [25]:
# Initialize the K-Means model. Define clusters before running
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([3, 3, 3, 3, 1, 3, 3, 0, 1, 0, 3, 1, 3, 3, 0, 3, 3, 3, 0, 1, 1, 3,
       0, 3, 3, 0, 3, 1, 0, 0, 3, 3, 3, 0, 3, 2, 0, 3, 3, 1, 1, 0, 3, 1,
       1, 0, 3, 1, 1, 1, 1, 0, 3, 3, 3, 3, 1, 3, 0, 0, 3, 0, 3, 3, 3, 0,
       1, 0, 1, 3, 3, 3, 0, 1, 0, 1, 1, 0, 3, 1, 3, 0, 3, 3, 0, 0, 3, 0,
       4, 2, 3, 0, 4, 3, 3, 3, 0, 3, 3, 0, 1, 0, 0, 0, 3, 0, 1, 0, 3, 1,
       3, 3, 3, 1, 0, 3, 3, 0, 1, 0, 3, 0, 0, 1, 0, 3, 0, 3, 1, 1, 1, 1,
       3, 3, 1, 1, 0, 3, 3, 0, 0, 0, 0, 1, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0,
       0, 0, 3, 3, 1, 3, 3, 3, 0, 0, 2, 0, 3, 3, 3, 1, 1, 1, 1, 1, 1, 0,
       0, 3, 3, 0, 0, 0, 3, 1, 1, 0, 3, 0, 3, 0, 1, 0, 0, 0, 0, 2, 3, 0,
       3, 1, 0, 4, 0, 0, 1, 3, 1, 1, 3, 3, 0, 1, 1], dtype=int32)

In [26]:
# Concatentate the worldbank_df and pcs_df DataFrames on the same columns.
merged_df= pd.merge(worldbank_df, pcs_df, left_index=True, right_index=True)
merged_df

,SP.RUR.TOTL.ZG,SP.DYN.LE00.MA.IN,SP.DYN.CDRT.IN,SE.SEC.AGES,TT.PRI.MRCH.XD.WD,TM.VAL.MRCH.HI.ZS,SP.URB.TOTL.IN.ZS,SP.POP.TOTL,NY.ADJ.AEDU.GN.ZS,TM.VAL.MRCH.WL.CD,...,TM.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TM.UVI.MRCH.XD.WD,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.MKTP.CN,PC1,PC2,PC3
0,1.165446,72.533619,8.004524,12.000000,107.455250,84.024361,44.087857,1.012436e+05,5.534013,1.087467e+09,...,4.783515,1.125688e+08,216.658854,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14,-1.458722,-1.883285,-0.504422
1,0.219893,67.868398,8.191021,11.829812,145.250894,71.802046,87.698762,5.704352e+04,11.760709,1.792657e+08,...,0.002126,4.184260e+07,143.513039,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14,-1.415482,-1.809381,-0.423181
2,-1.441178,77.380952,9.809524,12.000000,97.331406,84.902486,97.634000,1.088938e+07,5.853018,3.630866e+11,...,0.126156,3.792406e+11,159.111986,4.421130e+11,40354.739298,3.329685e+04,3.646237e+11,-4.289612,-0.851653,-0.802823
3,0.498354,76.756810,8.353000,11.000000,108.709459,82.995957,32.113905,2.808053e+05,5.273782,1.516150e+09,...,0.568932,3.391027e+08,156.621750,4.297310e+09,15267.202063,3.053440e+04,8.594619e+09,-2.219694,-1.981160,-0.642262
4,2.877665,59.468381,8.529857,13.000000,144.139332,13.475310,23.813571,2.976380e+07,2.633482,5.220148e+09,...,17.256922,4.615880e+08,135.988659,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14,5.584652,1.086053,-0.171742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,-0.939745,64.564762,15.342857,10.000000,97.418236,45.066431,68.436762,4.619452e+07,5.381339,4.894700e+10,...,0.344069,4.288368e+10,199.027240,1.165848e+11,2599.344442,3.400556e+04,1.480505e+12,-1.569696,-0.350591,0.618222
209,-2.673514,72.779762,9.386048,12.000000,106.540275,29.258109,94.139095,3.373326e+06,3.663434,7.075817e+09,...,4.982259,5.763423e+09,132.264065,3.875424e+10,11415.997493,2.947178e+05,1.003606e+12,-2.142622,-1.365432,-0.000238
210,1.192000,66.939524,5.033333,11.000000,156.182226,37.723131,49.581048,2.885206e+07,9.374764,1.363325e+10,...,0.006456,7.316529e+09,134.589546,4.345603e+10,1438.250604,4.762089e+06,1.520066e+14,0.628151,0.098670,2.218181
211,2.088479,52.597905,10.936762,14.000000,159.612380,29.054215,39.449000,1.393321e+07,2.556854,5.404199e+09,...,0.695017,5.549497e+09,144.208141,1.674368e+10,1144.799404,7.938876e+03,1.236047e+11,7.005627,1.533691,-0.813044


In [27]:
#  Add a new column, "Country" to the clustered_df DataFrame 
merged_df["Country"] = country_names_df["Country"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
merged_df["class"] = model.labels_
pcs_df.head()

# Print the shape of the clustered_df
print(merged_df.shape)
merged_df.head(10)

(213, 53)


,SP.RUR.TOTL.ZG,SP.DYN.LE00.MA.IN,SP.DYN.CDRT.IN,SE.SEC.AGES,TT.PRI.MRCH.XD.WD,TM.VAL.MRCH.HI.ZS,SP.URB.TOTL.IN.ZS,SP.POP.TOTL,NY.ADJ.AEDU.GN.ZS,TM.VAL.MRCH.WL.CD,...,TM.UVI.MRCH.XD.WD,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.MKTP.CN,PC1,PC2,PC3,Country,class
0,1.165446,72.533619,8.004524,12.000000,107.455250,84.024361,44.087857,1.012436e+05,5.534013,1.087467e+09,...,216.658854,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14,-1.458722,-1.883285,-0.504422,ABW,3
1,0.219893,67.868398,8.191021,11.829812,145.250894,71.802046,87.698762,5.704352e+04,11.760709,1.792657e+08,...,143.513039,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14,-1.415482,-1.809381,-0.423181,ASM,3
2,-1.441178,77.380952,9.809524,12.000000,97.331406,84.902486,97.634000,1.088938e+07,5.853018,3.630866e+11,...,159.111986,4.421130e+11,40354.739298,3.329685e+04,3.646237e+11,-4.289612,-0.851653,-0.802823,BEL,3
3,0.498354,76.756810,8.353000,11.000000,108.709459,82.995957,32.113905,2.808053e+05,5.273782,1.516150e+09,...,156.621750,4.297310e+09,15267.202063,3.053440e+04,8.594619e+09,-2.219694,-1.981160,-0.642262,BRB,3
4,2.877665,59.468381,8.529857,13.000000,144.139332,13.475310,23.813571,2.976380e+07,2.633482,5.220148e+09,...,135.988659,3.224885e+11,14968.403638,1.742575e+06,1.181116e+14,5.584652,1.086053,-0.171742,AFG,1
5,-0.026400,73.535000,3.883810,12.000000,162.687811,58.956652,74.883952,3.886098e+05,3.109797,2.711729e+09,...,130.857505,1.201073e+10,30524.819935,4.356931e+04,1.703543e+10,-1.414161,-1.307499,0.285723,BRN,3
6,-2.529609,74.298286,6.862714,10.590909,94.292866,70.920055,52.041143,2.943197e+06,3.111894,4.093322e+09,...,141.258817,1.056913e+10,3623.215444,3.914092e+05,1.141879e+12,-2.182922,-1.058004,0.939800,ALB,3
7,3.247449,67.868398,8.191021,12.000000,114.091846,60.336391,89.391667,7.797129e+04,2.403886,7.161529e+10,...,147.348997,3.005556e+09,38075.776620,3.137207e+04,2.456418e+09,-1.026563,-1.088662,-0.642386,AND,0
8,1.410318,52.250476,12.233714,11.363636,167.788793,58.421773,59.388571,2.381554e+07,3.211000,1.421981e+10,...,125.279736,7.537299e+10,2982.568601,3.965798e+05,1.093448e+13,5.604188,1.891779,-0.281637,AGO,1
9,1.024598,67.868398,8.191021,12.000000,114.091846,60.336391,89.950810,1.468935e+05,5.018118,7.161529e+10,...,147.348997,3.224885e+11,14968.403638,3.326798e+04,4.910242e+09,-0.932873,-0.829744,-0.138219,CUW,0


### Connect to AWS RDS Database

In [32]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://final-group-project-3.cm2mayhhsyby.us-east-1.rds.amazonaws.com:5432/final_project_group_3"
config = {"user":"postgres", 
          "password": "Ey34EcgtYcg8gGMR", 
          "driver":"org.postgresql.Driver"}

In [35]:
# Conver to a PySpark DF for import
merged_df = spark.createDataFrame(merged_df)

In [36]:
# Write merged_df to table in RDS
merged_df.write.jdbc(url=jdbc_url, table='worldbank_data_postprocessed', mode=mode, properties=config)

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(merged_df, x='PC1', y='PC2', z='PC3',color='class', hover_name='Country', hover_data=['SP.DYN.LE00.MA.IN'])
fig.show()

In [ ]:
# Scaling data to create the scatter plot
X_cluster = merged_df[['Country', 'SP.DYN.LE00.MA.IN']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(X_cluster_scaled, columns=['x', y'], index=clustered_df.index)
plot_df.head()

In [ ]:
# Create a hvplot.scatter plot using x="x" and y="y".
plot_df.hvplot.scatter(
    x="x",
    y="y",
    hover_cols=["Define"],
    by="Class",
)
